In [46]:
!pip install pyspark

In [47]:
import pandas as pd
import numpy as np
import pyspark
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('Uber').getOrCreate()

In [49]:
from pyspark.sql.functions import trim

df=spark.read.csv("dataset.csv",header=True,inferSchema=True)
trimmed_column_names = [col_name.strip() for col_name in df.columns]

# Create a dictionary to map the current column names to the trimmed column names
column_rename_mapping = dict(zip(df.columns, trimmed_column_names))

# Rename the columns in the DataFrame
for old_col_name, new_col_name in column_rename_mapping.items():
    df = df.withColumnRenamed(old_col_name, new_col_name)

# Show the DataF

df.show(1,vertical=True)


-RECORD 0--------------------
 Date            | 10-Sep-12 
 Time (Local)    | 7         
 Eyeballs        | 5         
 Zeroes          | 0         
 Completed Trips | 2         
 Requests        | 2         
 Unique Drivers  | 9         
only showing top 1 row



In [50]:
from datetime import datetime ,timedelta
import random

# Function to generate random dates
def generate_random_date(start_date, end_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    end_datetime = datetime.strptime(end_date, "%Y-%m-%d")
    delta = end_datetime - start_datetime
    random_days = random.randint(0, delta.days)
    random_date = start_datetime + timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

# Define the range for random dates (you can adjust this range as needed)
start_date='2023-01-01'
end_date='2024-06-01'


In [51]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import IntegerType
import random

# Define the generate_random_time function before it's used in the UDF
def generate_random_time():
    return random.randint(1, 86400) 

# Register the UDF
random_time_udf = udf(generate_random_time, IntegerType())

# Apply the UDF to fill null values in the "Time (Local)" column
df = df.withColumn("Time (Local)", when(col("Time (Local)").isNull(), random_time_udf()).otherwise(col("Time (Local)")))


In [52]:
from pyspark.sql.functions import udf,when,col


# Replace null values in the "Date" column with random dates
random_date_udf = udf(lambda: generate_random_date(start_date,end_date))
df = df.withColumn("Date", when(col("Date").isNull(), random_date_udf()).otherwise(col("Date")))

df.show()

+----------+------------+--------+------+---------------+--------+--------------+
|      Date|Time (Local)|Eyeballs|Zeroes|Completed Trips|Requests|Unique Drivers|
+----------+------------+--------+------+---------------+--------+--------------+
| 10-Sep-12|           7|       5|     0|              2|       2|             9|
|2023-08-31|           8|       6|     0|              2|       2|            14|
|2024-04-14|           9|       8|     3|              0|       0|            14|
|2023-12-31|          10|       9|     2|              0|       1|            14|
|2023-01-05|          11|      11|     1|              4|       4|            11|
|2023-03-11|          12|      12|     0|              2|       2|            11|
|2023-06-02|          13|       9|     1|              0|       0|             9|
|2023-09-23|          14|      12|     1|              0|       0|             9|
|2024-03-17|          15|      11|     2|              1|       2|             7|
|2023-10-17|    

# Daily Trip Trends

In [53]:
from pyspark.sql.functions import sum,to_date
# Convert Date column to DateType
df = df.withColumn("Date",to_date("Date"))

daily_trip_trends = df.groupBy("Date").agg(sum("Completed Trips").alias("Total Completed Trips ")) \
    .orderBy("Date")

daily_trip_trends.show()




+----------+----------------------+
|      Date|Total Completed Trips |
+----------+----------------------+
|      NULL|                   104|
|2023-01-01|                     4|
|2023-01-02|                     1|
|2023-01-03|                     5|
|2023-01-06|                     2|
|2023-01-10|                     0|
|2023-01-11|                    13|
|2023-01-13|                     0|
|2023-01-15|                     2|
|2023-01-16|                     1|
|2023-01-17|                    12|
|2023-01-20|                     1|
|2023-01-21|                     5|
|2023-01-25|                     5|
|2023-01-26|                    25|
|2023-01-27|                     1|
|2023-01-30|                     2|
|2023-02-04|                     0|
|2023-02-05|                     0|
|2023-02-07|                     0|
+----------+----------------------+
only showing top 20 rows



# Peak Hours Analysis:

In [54]:
from pyspark.sql.functions import hour,from_unixtime
df = df.withColumn("Time (Local)", from_unixtime("Time (Local)"))

null_count = df.filter(df["Time (Local)"].isNull()).count()

print(null_count)

peak_hours_analysis = df \
    .withColumn("Hour", hour("Time (Local)")) \
    .groupBy("Hour") \
    .agg(sum("Completed Trips").alias("Total Completed Trips")) \
    .orderBy("Total Completed Trips", ascending=False)

peak_hours_analysis.show()



0
+----+---------------------+
|Hour|Total Completed Trips|
+----+---------------------+
|   5|                 1365|
+----+---------------------+



# Weekday vs. Weekend Comparison

In [56]:
from pyspark.sql.functions import dayofweek

weekday_vs_weekend = df.withColumn("DayOfWeek",dayofweek("Date")) \
    .withColumn("Weekend",when(col("DayOfWeek").isin(1,7),"Weekend").otherwise("Weekday")) \
    .groupBy("Weekend").agg(sum("Completed Trips")).alias("Total Completed Trips")



weekday_vs_weekend.show()

+-------+--------------------+
|Weekend|sum(Completed Trips)|
+-------+--------------------+
|Weekday|                1002|
|Weekend|                 363|
+-------+--------------------+



# Request Fulfillment Rate:

In [70]:
from pyspark.sql.functions import sum

request_fulfillment_rate = df.agg((sum("Completed Trips")/sum("Requests")).alias("Request Fulfillment Rate"))

request_fulfillment_rate.show()

+------------------------+
|Request Fulfillment Rate|
+------------------------+
|      0.7346609257265877|
+------------------------+



# Driver Utilization:


In [58]:
from pyspark.sql.functions import countDistinct, sum

# Calculate the total completed trips using sum aggregation
total_completed_trips = df.groupBy("Date").agg(sum("Completed Trips").alias("Total Completed Trips"))

# Calculate the utilization rate using the total completed trips and count of distinct unique drivers
driver_utilization = total_completed_trips.join(
    df.groupBy("Date").agg(countDistinct("Unique Drivers").alias("Unique Drivers")),
    "Date"
).withColumn("Utilization Rate", col("Total Completed Trips") / col("Unique Drivers"))

driver_utilization.show()



+----------+---------------------+--------------+----------------+
|      Date|Total Completed Trips|Unique Drivers|Utilization Rate|
+----------+---------------------+--------------+----------------+
|2024-05-30|                    9|             1|             9.0|
|2023-05-22|                    2|             1|             2.0|
|2023-02-25|                    7|             1|             7.0|
|2024-04-20|                    0|             1|             0.0|
|2024-01-11|                    0|             1|             0.0|
|2023-09-27|                    0|             1|             0.0|
|2023-06-24|                    0|             2|             0.0|
|2023-10-12|                    0|             1|             0.0|
|2024-02-16|                    0|             1|             0.0|
|2023-05-16|                    2|             1|             2.0|
|2023-06-01|                   14|             1|            14.0|
|2023-02-16|                    0|             1|             

# Eyeballs vs. Completed Trips:

In [60]:
eyeballs_vs_completed = df.groupBy("Eyeballs").agg(sum("Completed Trips").alias("Total Completed Trips"))


eyeballs_vs_completed.show()

+--------+---------------------+
|Eyeballs|Total Completed Trips|
+--------+---------------------+
|      31|                   15|
|      53|                   28|
|      34|                   45|
|      28|                   24|
|      26|                   22|
|      27|                    9|
|      44|                   17|
|      12|                    8|
|      22|                   36|
|      47|                   16|
|       1|                    0|
|      13|                   11|
|       6|                    6|
|      16|                   21|
|       3|                    3|
|      20|                   29|
|      40|                   20|
|      94|                   36|
|       5|                    8|
|      19|                   39|
+--------+---------------------+
only showing top 20 rows



# Zeroes Analysis:

In [61]:
zeroes_analysis = df.filter(col("Zeroes")!=0).count()

print("Number of non-zero entries in 'Zeroes':",zeroes_analysis)

Number of non-zero entries in 'Zeroes': 310


# Request Patterns by Time:

In [62]:
from pyspark.sql.functions import hour,count

request_patterns_by_time = df.groupBy(hour("Time (Local)")).agg(count("Requests").alias("Total Requests")) \
                            .orderBy("Total Requests",ascending=False)

request_patterns_by_time.show()


+------------------+--------------+
|hour(Time (Local))|Total Requests|
+------------------+--------------+
|                 5|           336|
+------------------+--------------+



# Generate random latitude and longitude values for each date

In [68]:
from pyspark.sql.functions import lit

df = df.withColumn("Latitude", lit(random.uniform(-90, 90))) \
                     .withColumn("Longitude", lit(random.uniform(-180, 180)))

# Show the DataFrame with generated latitude and longitude values
df.show(1,vertical=True)

-RECORD 0------------------------------
 Date            | NULL                
 Time (Local)    | 1970-01-01 05:30:07 
 Eyeballs        | 5                   
 Zeroes          | 0                   
 Completed Trips | 2                   
 Requests        | 2                   
 Unique Drivers  | 9                   
 Latitude        | -75.48375177280542  
 Longitude       | 38.4630733315166    
only showing top 1 row



# Geographical Analysis:

In [69]:
geo_analysis = df.groupBy("Latitude","Longitude").agg(sum("Completed Trips").alias("Total Completed Trips")) \
                .orderBy("Total Completed Trips", ascending=False)

geo_analysis.show()

+------------------+----------------+---------------------+
|          Latitude|       Longitude|Total Completed Trips|
+------------------+----------------+---------------------+
|-75.48375177280542|38.4630733315166|                 1365|
+------------------+----------------+---------------------+



# Trend Analysis Over Time:

In [71]:
trend_analysis_over_time = df.groupBy("Date").agg(sum("Completed Trips").alias("Total Completed Trips"),
                                                  sum("Requests").alias("Total Requests"),
                                                  countDistinct("Unique Drivers").alias("Unique Drivers")) \
                                                  .orderBy("Date")



trend_analysis_over_time.show()
                                                  


+----------+---------------------+--------------+--------------+
|      Date|Total Completed Trips|Total Requests|Unique Drivers|
+----------+---------------------+--------------+--------------+
|      NULL|                  104|           144|             9|
|2023-01-07|                    9|             9|             1|
|2023-01-10|                    0|             2|             1|
|2023-01-11|                    3|             5|             2|
|2023-01-13|                    7|            10|             1|
|2023-01-14|                    6|             7|             1|
|2023-01-18|                    1|             2|             1|
|2023-01-19|                    0|             0|             1|
|2023-01-21|                    3|             4|             1|
|2023-01-22|                    8|             9|             1|
|2023-01-24|                    3|             3|             1|
|2023-01-27|                    2|             4|             1|
|2023-01-29|             